In [1]:
import numpy as np
from scipy.integrate import solve_ivp

# Define the Hill function for regulation
# Source [5]: f(u, K) = (u/K)^H / (1 + (u/K)^H) for activator
# Source [5]: f(u, K) = 1 - (u/K)^H / (1 + (u/K)^H) for repressor
def f_reg(u, k, h, sign):
    """Hill function for activation or repression."""
    # Ensure u is non-negative to avoid issues with fractional exponents if H is not an integer
    u = np.maximum(0, u)
    if sign == 1:  # Activation
        return (u / k)**h / (1 + (u / k)**h)
    elif sign == -1: # Repression
        return 1 - (u / k)**h / (1 + (u / k)**h)
    else:
        raise ValueError("Sign must be 1 (activation) or -1 (repression)")

# Define the competitive binding function for OR gate logic
# Source [5]: fc(u; Ku, Kv, v) where u regulates at Ku, v regulates at Kv
# Source [5]: For activator u: (u/Ku)^H / (1 + (u/Ku)^H + (v/Kv)^H)
# Source [5]: For repressor u: 1 - (u/Ku)^H / (1 + (u/Ku)^H + (v/Kv)^H)
def fc_reg(u, ku, kv, v, h, sign):
    """Competitive binding function for OR gate."""
    # Ensure inputs are non-negative
    u = np.maximum(0, u)
    v = np.maximum(0, v)
    
    # Handle potential division by zero if K values are zero, though typically K > 0
    u_term = (u / ku)**h if ku != 0 else float('inf') if u > 0 else 0
    v_term = (v / kv)**h if kv != 0 else float('inf') if v > 0 else 0
    
    denominator = 1 + u_term + v_term

    if sign == 1: # Activation
        return u_term / denominator
    elif sign == -1: # Repression
        # The source formula is 1 - (u/Ku)^H / (1 + (u/Ku)^H + (v/Kv)^H) [5]
        # This form seems unusual for repression; however, we follow the source directly.
        return 1 - u_term / denominator
    else:
        raise ValueError("Sign must be 1 (activation) or -1 (repression)")


# Define the G gate function for Z regulation
# Source [5]: AND-gate is Gz = f(X*, Kxz)f(Y*, Kyz)
# Source [5]: OR-gate is Gz = fc(X*; Kxz, Kyz, Y*) + fc(Y*; Kyz, Kxz, X*)
def G_gate(active_x, k_xz, sign_xz, active_y, k_yz, sign_yz, h, gate_logic):
    """Gate function for Z regulation (AND or OR logic)."""
    if gate_logic == 'AND':
        # Source [5] defines AND using product of f functions
        return f_reg(active_x, k_xz, h, sign_xz) * f_reg(active_y, k_yz, h, sign_yz)
    elif gate_logic == 'OR':
        # Source [5] defines OR using sum of fc functions (competitive binding)
        # Note: The fc_reg function takes Ku, Kv, and v.
        # For X* regulating Z, Ku is K_xz, Kv is K_yz, and v is active_y
        # For Y* regulating Z, Ku is K_yz, Kv is K_xz, and v is active_x
        return fc_reg(active_x, k_xz, k_yz, active_y, h, sign_xz) + \
               fc_reg(active_y, k_yz, k_xz, active_x, h, sign_yz)
    else:
        raise ValueError("Gate logic must be 'AND' or 'OR'")

# Mapping FFL type strings to signs (sign_XY, sign_XZ, sign_YZ)
# Based on structures shown in Tables 1 and 2 [6, 7]
FFL_SIGNS = {
    'coherent_type1': (1, 1, 1),
    'coherent_type2': (1, -1, -1),
    'coherent_type3': (-1, 1, -1),
    'coherent_type4': (-1, -1, 1),
    'incoherent_type1': (1, 1, -1),
    'incoherent_type2': (1, -1, 1),
    'incoherent_type3': (-1, 1, 1),
    'incoherent_type4': (-1, -1, -1),
}

# Define the system of ODEs for Y and Z concentrations
# Source [5]: dY/dt = By + αy f(X*, Kxy) - βy Y
# Source [5]: dZ/dt = Bz + αz G(X*, Kxz,Y*, Kyz) - βz Z
# Source [8]: X is constitutively produced at level 1. X* = X if Sx=1, 0 if Sx=0 -> X* = 1 * Sx
# Source [8]: Y* = Y if Sy=1, 0 if Sy=0 -> Y* = Y * Sy (Y here is the concentration of protein Y)
def system_odes(t, y_z, params, sx_func, sy_func):
    """System of ODEs for FFL simulation."""
    y, z = y_z  # y_z contains [concentration of Y, concentration of Z]

    # Get current input signals (assuming Sx and Sy are binary 0 or 1)
    sx_state = sx_func(t, params)
    sy_state = sy_func(t, params)

    # Determine active forms of X and Y based on inputs [8]
    # X_total is assumed constitutively 1 [8]
    active_x = 1.0 * sx_state
    # Y* is the concentration of Y gated by Sy [8]
    active_y = y * sy_state

    # Get parameters
    alpha_y = params['alpha_y']
    beta_y = params['beta_y']
    basal_y = params['basal_y']
    k_xy = params['K_xy']
    
    alpha_z = params['alpha_z']
    beta_z = params['beta_z']
    basal_z = params['basal_z']
    k_xz = params['K_xz']
    k_yz = params['K_yz']
    
    h = params['H']
    sign_xy = params['sign_XY']
    sign_xz = params['sign_XZ']
    sign_yz = params['sign_YZ']
    gate_logic = params['gate_logic']
    
    is_simple = params.get('is_simple_regulation', False) # Default is False
    simple_y_level = params.get('simple_Y_level', 1.0) # Default Y=1 for simple

    # Calculate rates of change
    if is_simple:
        # Simple regulation: Y is constitutively active at a fixed level [9]
        dy_dt = 0 # Y concentration does not change as it's constitutively present/active
        
        # In simple regulation, the source diagram [1b] and description [9] imply
        # active_Y is simply the constitutive level (e.g. 1), ungated by Sy.
        # Let's use the defined simple_Y_level parameter.
        simple_active_y = simple_y_level
        
        # Z is regulated by X and the constitutive Y
        dz_dt = basal_z + alpha_z * G_gate(active_x, k_xz, sign_xz, simple_active_y, k_yz, sign_yz, h, gate_logic) - beta_z * z
        
    else:
        # FFL regulation
        dy_dt = basal_y + alpha_y * f_reg(active_x, k_xy, h, sign_xy) - beta_y * y
        dz_dt = basal_z + alpha_z * G_gate(active_x, k_xz, sign_xz, active_y, k_yz, sign_yz, h, gate_logic) - beta_z * z

    return [dy_dt, dz_dt]

# Main simulation function
def simulate_ffl(ffl_type, gate_logic, t_span, t_eval, initial_conditions, parameters, sx_func, sy_func):
    """
    Simulates the dynamics of an FFL or simple regulation circuit.

    Args:
        ffl_type (str or None): The type of FFL ('coherent_type1', 'incoherent_type4', etc.).
                                If None, simulates simple regulation.
        gate_logic (str): 'AND' or 'OR' for the Z promoter.
        t_span (tuple): (start_time, end_time) for the simulation.
        t_eval (array): Specific time points at which to store the solution.
        initial_conditions (list): Initial concentrations [Y_0, Z_0].
                                   For simple regulation, initial_conditions is ignored.
        parameters (dict): Dictionary of biochemical parameters (alpha_y, beta_y, etc.).
                           Requires H, K_xy, K_xz, K_yz.
                           Basal rates (basal_y, basal_z) default to 0 if not provided.
                           Alpha rates (alpha_y, alpha_z) default to 1 if not provided.
                           Beta rates (beta_y, beta_z) default to 1 if not provided.
                           For simple regulation (ffl_type is None), requires simple_Y_level.
        sx_func (function): Function that takes (t, params) and returns the state of Sx (0 or 1) at time t.
        sy_func (function): Function that takes (t, params) and returns the state of Sy (0 or 1) at time t.

    Returns:
        tuple: (t_eval, results), where t_eval are the time points and results
               is a numpy array of shape (len(t_eval), 2) with concentrations
               [Y, Z] at each time point.
    """
    if ffl_type is None:
        # Simple regulation case
        parameters['is_simple_regulation'] = True
        # Check required parameter for simple regulation
        if 'simple_Y_level' not in parameters:
             raise ValueError("Parameters must include 'simple_Y_level' for simple regulation.")
        # Signs are still used by the G_gate function even in simple regulation,
        # representing the type of regulation X and Y have on Z.
        # For comparison to FFLs, you'd choose signs matching the Z regulation in the FFL.
        # Let's require sign_XZ, sign_YZ, and gate_logic even for simple regulation
        if not all(k in parameters for k in ['sign_XZ', 'sign_YZ', 'gate_logic']):
             raise ValueError("Parameters for simple regulation must include 'sign_XZ', 'sign_YZ', and 'gate_logic'.")
        
        # The ODE solver needs an initial condition for Y, but it won't change in simple mode
        # Set initial Y to the simple_Y_level
        initial_conditions = parameters['simple_Y_level'] 

    else:
        # FFL case
        parameters['is_simple_regulation'] = False
        
        # Get signs based on FFL type string
        if ffl_type not in FFL_SIGNS:
            raise ValueError(f"Unknown FFL type: {ffl_type}. Choose from {list(FFL_SIGNS.keys())}.")
        
        parameters['sign_XY'], parameters['sign_XZ'], parameters['sign_YZ'] = FFL_SIGNS[ffl_type]

    # Set default parameter values if not provided
    params_with_defaults = {
        'alpha_y': parameters.get('alpha_y', 1.0),
        'beta_y': parameters.get('beta_y', 1.0),
        'basal_y': parameters.get('basal_y', 0.0),
        'alpha_z': parameters.get('alpha_z', 1.0),
        'beta_z': parameters.get('beta_z', 1.0),
        'basal_z': parameters.get('basal_z', 0.0),
        'K_xy': parameters.get('K_xy'),
        'K_xz': parameters.get('K_xz'),
        'K_yz': parameters.get('K_yz'),
        'H': parameters.get('H'),
        'gate_logic': gate_logic,
        'is_simple_regulation': parameters['is_simple_regulation'],
        'simple_Y_level': parameters.get('simple_Y_level', None), # Keep simple_Y_level in params
        'sign_XY': parameters.get('sign_XY', None), # Keep signs in params
        'sign_XZ': parameters.get('sign_XZ', None),
        'sign_YZ': parameters.get('sign_YZ', None),
    }
    
    # Check required parameters for FFL
    if not params_with_defaults['is_simple_regulation']:
         if not all(k in params_with_defaults for k in ['K_xy', 'K_xz', 'K_yz', 'H']):
             raise ValueError("Parameters for FFL must include 'K_xy', 'K_xz', 'K_yz', and 'H'.")
    # Check required parameters for Simple (already done above, but double-check H)
    if params_with_defaults['is_simple_regulation']:
         if not all(k in params_with_defaults for k in ['K_xz', 'K_yz', 'H', 'simple_Y_level']):
              raise ValueError("Parameters for simple regulation must include 'K_xz', 'K_yz', 'H', and 'simple_Y_level'.")


    # Solve the ODEs
    # We use dense_output=True which is more efficient if you need solutions
    # at many time points using t_eval
    result = solve_ivp(
        lambda t, y_z: system_odes(t, y_z, params_with_defaults, sx_func, sy_func),
        t_span,
        initial_conditions,
        t_eval=t_eval,
        method='LSODA' # A good general-purpose method
    )

    if not result.success:
        print(f"Warning: ODE solver failed with status {result.status}: {result.message}")

    return result.t, result.y.T # Transpose y to get [time_points, concentrations]

# --- Example Usage ---

# Define simple step functions for inputs Sx and Sy
def sx_step_on(t, params):
    """Sx steps from 0 to 1 at t_step."""
    t_step = params.get('sx_step_time', 10)
    return 1.0 if t >= t_step else 0.0

def sy_always_on(t, params):
    """Sy is always 1."""
    return 1.0

def sy_always_off(t, params):
    """Sy is always 0."""
    return 0.0

# Example 1: Simulate Coherent Type 1 AND FFL response to Sx step (Sy always on)
print("Simulating Coherent Type 1 AND FFL...")
t_span = (0, 50) # Time from 0 to 50 life times (assuming beta=1 means life time = 1) [5, 10]
t_eval = np.linspace(t_span, t_span[1], 200)
initial_conditions = [0.0, 0.0] # Start with 0 concentration of Y and Z

# Parameters based loosely on source [11] defaults (H=2, alpha=1, beta=1, basal=0)
# K values chosen to be within a functional range [9]
ffl_parameters = {
    'H': 2,
    'K_xy': 0.1, # Activation coefficient of Y by X
    'K_xz': 0.1, # Activation coefficient of Z by X
    'K_yz': 0.5, # Activation coefficient of Z by Y (Example value from Fig 2 Left)
    'alpha_y': 1.0, 'beta_y': 1.0, 'basal_y': 0.0,
    'alpha_z': 1.0, 'beta_z': 1.0, 'basal_z': 0.0,
    'sx_step_time': 5, # Time when Sx steps on
}

t_coherent_AND, results_coherent_AND = simulate_ffl(
    ffl_type='coherent_type1',
    gate_logic='AND',
    t_span=t_span,
    t_eval=t_eval,
    initial_conditions=initial_conditions,
    parameters=ffl_parameters,
    sx_func=sx_step_on,
    sy_func=sy_always_on
)

y_coherent_AND = results_coherent_AND[:, 0]
z_coherent_AND = results_coherent_AND[:, 1]

print("Simulation complete for Coherent Type 1 AND FFL.")
# print(f"Final Y: {y_coherent_AND[-1]:.4f}, Final Z: {z_coherent_AND[-1]:.4f}") # Example output

# Example 2: Simulate Simple Regulation (for comparison to Coherent Type 1 AND)
# Source [9]: Simple regulation with Y constitutively expressed, Y = 1.
# Source [11]: Simple regulation parameters: Y=1, H=2, alpha_z=1, beta_z=1, Bz=0, Kxz=1.
# For a valid comparison, the simple system should ideally have the same steady-state Z level as the FFL [12, 13].
# This often requires adjusting Kxz or alpha_z in the simple system [13].
# Let's use the simple parameters given in [11] for the simple system, and assume the Z regulation is
# the direct X->Z part and the constitutive Y->Z part, integrated by the *same* gate logic as the FFL.
# In the case of Coherent Type 1 AND, both X->Z and Y->Z are activators.
print("\nSimulating Simple Regulation (comparison)...")

simple_parameters = {
    'H': 2,
    'K_xz': 1.0, # Activation coefficient of Z by X (from [11] for simple)
    'K_yz': ffl_parameters['K_yz'], # Use same Kyz as FFL for Y's effect
    'alpha_z': 1.0, 'beta_z': 1.0, 'basal_z': 0.0,
    'simple_Y_level': 1.0, # Constitutive level of Y [9, 11]
    'sx_step_time': 5, # Time when Sx steps on
    # Simple system needs signs for X->Z and Y->Z regulation types to use G_gate
    # Match the FFL type we compared against (Coherent Type 1)
    'sign_XZ': 1, # X->Z activation
    'sign_YZ': 1, # Y->Z activation
}

# Initial Y is set to simple_Y_level internally by the function for simple regulation
initial_conditions_simple = [simple_parameters['simple_Y_level'], 0.0] 

t_simple_reg, results_simple_reg = simulate_ffl(
    ffl_type=None, # Indicate simple regulation
    gate_logic='AND', # Use the same gate logic as the FFL comparison
    t_span=t_span,
    t_eval=t_eval,
    initial_conditions=initial_conditions_simple,
    parameters=simple_parameters,
    sx_func=sx_step_on,
    sy_func=sy_always_on # Sy state might not matter if Y is constitutive
)

y_simple_reg = results_simple_reg[:, 0] # Should be constant simple_Y_level
z_simple_reg = results_simple_reg[:, 1]

print("Simulation complete for Simple Regulation.")
# print(f"Final Y: {y_simple_reg[-1]:.4f}, Final Z: {z_simple_reg[-1]:.4f}")

# Note: To visually compare the delay/acceleration as described in the source [12, 13],
# you would typically plot the Z trajectories from both simulations and look at
# the time it takes to reach 50% of the steady-state level [10]. The parameters
# might need tuning to ensure similar steady-state levels for a controlled comparison [13].
# The parameters used here are just examples; for direct comparison studies,
# you'd need to adjust parameters to match steady states.

# Example 3: Simulate Incoherent Type 1 AND FFL response to Sx step (Sy always on), with basal Y
print("\nSimulating Incoherent Type 1 AND FFL (with basal Y)...")
# This type with basal Y acts as a sign-sensitive accelerator [13, 14]
incoherent_params_basal = {
    'H': 2,
    'K_xy': 1.0,  # Source Fig 4 Left uses Kxy=1
    'K_xz': 1.0,  # Source Fig 4 Left uses Kxz=1
    'K_yz': 0.5,  # Source Fig 4 Left uses Kyz=0.5
    'alpha_y': 1.0, 'beta_y': 1.0, 'basal_y': 0.5, # Example basal_y from Fig 4 Left {0.5, 0.3}
    'alpha_z': 1.0, 'beta_z': 1.0, 'basal_z': 0.0,
    'sx_step_time': 5,
}

t_incoherent_basal, results_incoherent_basal = simulate_ffl(
    ffl_type='incoherent_type1',
    gate_logic='AND',
    t_span=t_span,
    t_eval=t_eval,
    initial_conditions=initial_conditions, # Start Y and Z at 0
    parameters=incoherent_params_basal,
    sx_func=sx_step_on,
    sy_func=sy_always_on
)

y_incoherent_basal = results_incoherent_basal[:, 0]
z_incoherent_basal = results_incoherent_basal[:, 1]

print("Simulation complete for Incoherent Type 1 AND FFL (with basal Y).")
# print(f"Final Y: {y_incoherent_basal[-1]:.4f}, Final Z: {z_incoherent_basal[-1]:.4f}")


# Example 4: Simulate Incoherent Type 4 AND FFL pulse (no basal Y, Sy enabled)
print("\nSimulating Incoherent Type 4 AND FFL (pulse, Sy enabled)...")
# This type with no basal Y acts as a pulser gated by Sy [15]
incoherent_params_pulse = {
    'H': 2,
    'K_xy': 0.1,  # Source Fig 3 Right uses Kxy=0.1
    'K_xz': 0.1,  # Source Fig 3 Right uses Kxz=0.1
    'K_yz': 1.0,  # Source Fig 3 Right uses Kyz={1, 0.3, 0.1} (using 1 for strong pulse example)
    'alpha_y': 1.0, 'beta_y': 1.0, 'basal_y': 0.0, # No basal Y
    'alpha_z': 1.0, 'beta_z': 1.0, 'basal_z': 0.0,
    'sx_step_time': 5,
}

t_incoherent_pulse, results_incoherent_pulse = simulate_ffl(
    ffl_type='incoherent_type4',
    gate_logic='AND',
    t_span=t_span,
    t_eval=t_eval,
    initial_conditions=initial_conditions, # Start Y and Z at 0
    parameters=incoherent_params_pulse,
    sx_func=sx_step_on,
    sy_func=sy_always_on # Sy enabled (on)
)

y_incoherent_pulse = results_incoherent_pulse[:, 0]
z_incoherent_pulse = results_incoherent_pulse[:, 1]

print("Simulation complete for Incoherent Type 4 AND FFL (pulse, Sy enabled).")
# print(f"Final Y: {y_incoherent_pulse[-1]:.4f}, Final Z: {z_incoherent_pulse[-1]:.4f}")

# To visualize these results, you would typically use a plotting library like matplotlib.
# import matplotlib.pyplot as plt
# plt.figure()
# plt.plot(t_coherent_AND, z_coherent_AND, label='Coherent Type 1 AND (FFL)')
# plt.plot(t_simple_reg, z_simple_reg, label='Simple Regulation')
# plt.xlabel('Time [Life Time]')
# plt.ylabel('Z Concentration')
# plt.title('Coherent FFL Delay (Example)')
# plt.legend()
# plt.grid(True)
# plt.show()

# plt.figure()
# plt.plot(t_incoherent_basal, z_incoherent_basal, label='Incoherent Type 1 AND (Basal Y)')
# plt.xlabel('Time [Life Time]')
# plt.ylabel('Z Concentration')
# plt.title('Incoherent FFL Acceleration (Example)')
# plt.legend()
# plt.grid(True)
# plt.show()

# plt.figure()
# plt.plot(t_incoherent_pulse, z_incoherent_pulse, label='Incoherent Type 4 AND (Pulse, Sy=1)')
# # You might also simulate with Sy_always_off to show pulse is disabled
# # t_inc_pulse_sy_off, results_inc_pulse_sy_off = simulate_ffl('incoherent_type4', 'AND', t_span, t_eval, initial_conditions, incoherent_params_pulse, sx_step_on, sy_always_off)
# # plt.plot(t_inc_pulse_sy_off, results_inc_pulse_sy_off[:, 1], label='Incoherent Type 4 AND (Pulse, Sy=0)', linestyle='--')
# plt.xlabel('Time [Life Time]')
# plt.ylabel('Z Concentration')
# plt.title('Incoherent FFL Pulse (Example)')
# plt.legend()
# plt.grid(True)
# plt.show()

Simulating Coherent Type 1 AND FFL...


ValueError: `t_eval` must be 1-dimensional.